In [1]:
import seaborn as sns

data = sns.load_dataset('tips')

In [2]:
data.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
data['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [4]:
data['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [5]:
data.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
lb = LabelEncoder()

In [8]:
data['time'] = lb.fit_transform(data['time'])

In [9]:
X = data.drop(labels='time',axis=1)
y = data['time']

In [10]:
X

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [11]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [12]:
from sklearn.model_selection import train_test_split


In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [14]:
X_train

,total_bill,tip,sex,smoker,day,size
228,13.28,2.72,Male,No,Sat,2
208,24.27,2.03,Male,Yes,Sat,2
96,27.28,4.00,Male,Yes,Fri,2
167,31.71,4.50,Male,No,Sun,4
84,15.98,2.03,Male,No,Thur,2
...,...,...,...,...,...,...
106,20.49,4.06,Male,Yes,Sat,2
14,14.83,3.02,Female,No,Sun,2
92,5.75,1.00,Female,Yes,Fri,2
179,34.63,3.55,Male,Yes,Sun,2


In [15]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer

In [16]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195 entries, 228 to 102
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  195 non-null    float64 
 1   tip         195 non-null    float64 
 2   sex         195 non-null    category
 3   smoker      195 non-null    category
 4   day         195 non-null    category
 5   size        195 non-null    int64   
dtypes: category(3), float64(2), int64(1)
memory usage: 7.1 KB


In [17]:
cat_col = ['sex','smoker','day']
num_col = ['tip','total_bill','size']

In [18]:
num_pipeline = Pipeline(
steps=[
    ('impute',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])

cat_pipeline = Pipeline(
steps=[
    ('impute',SimpleImputer(strategy='most_frequent')),
    ('encode',OneHotEncoder())
])

In [19]:
preprocessor = ColumnTransformer([
    ('num pipeline',num_pipeline,num_col),
    ('cat pipeline',cat_pipeline,cat_col)
])

In [20]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [21]:
X_train[0]

array([-0.2580329 , -0.79306155, -0.61214068,  0.        ,  1.        ,
        1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ])

In [22]:
#Model building

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [32]:
models = {
    'RF':RandomForestClassifier(),
    'LR':LogisticRegression(),
    'DT':DecisionTreeClassifier()
}



In [37]:
from sklearn.metrics import accuracy_score

def evaluate_model(X_train,y_train,X_test,y_test,models):
    report={}
    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(X_train,y_train)
        
        y_pred = model.predict(X_test)
        
        accuracy = round(accuracy_score(y_test,y_pred)*100,2)
        
        report[list(models.keys())[i]]=accuracy
        
    return report

In [38]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'RF': 95.92, 'LR': 100.0, 'DT': 93.88}

In [68]:

params = {
    'n_estimators':[100,50],
    "criterion":['gini','entropy'],
    'max_depth':[3,5,10]
}

In [69]:
model=RandomForestClassifier(oob_score=True)

In [70]:
from sklearn.model_selection import RandomizedSearchCV

In [71]:
cv = RandomizedSearchCV(model,param_distributions=params,scoring='accuracy',cv=5,verbose=3)

In [72]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.949 total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.974 total time=   0.2s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.923 total time=   0.2s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=100;, score=0.949 total time=   0.3s
[CV 1/5] END criterion=entropy, max_depth=5, n_estimators=50;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=5, n_estimators=50;, score=0.923 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=5, n_estimators=50;, score=0.974 total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, n_estimators=50;, score=0.923 total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=5, n_estimators=50;, score=0.949 total time=  

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(oob_score=True),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10],
                                        'n_estimators': [100, 50]},
                   scoring='accuracy', verbose=3)

In [73]:
cv.best_params_

{'n_estimators': 100, 'max_depth': 5, 'criterion': 'gini'}

In [79]:
bestmodle = RandomForestClassifier(oob_score=True,n_estimators=100,max_depth=5,criterion='gini')

In [80]:
bestmodle.fit(X_train,y_train)

RandomForestClassifier(max_depth=5, oob_score=True)

In [81]:
y_ped = bestmodle.predict(X_test)

In [82]:
best_acc = accuracy_score(y_test,y_ped)

In [83]:
best_acc

0.9591836734693877